
# Cuadernillo: EfficientNet (Teoría + Implementación en PyTorch)

Este cuadernillo combina la explicación teórica de EfficientNet con una implementación educativa en PyTorch de la versión B0 (modelo base).

## Contenido:
- Introducción a EfficientNet y el escalado compuesto.
- Arquitectura de EfficientNet-B0.
- Implementación desde cero del bloque MBConv con Squeeze-and-Excitation (SE).
- Construcción de EfficientNet-B0 mini.
- Ejemplo de inferencia.
- Ejemplo de fine-tuning.


## 1. Introducción a EfficientNet

EfficientNet es una familia de redes neuronales convolucionales que introduce el concepto de **escalado compuesto** para mejorar el rendimiento y la eficiencia. La idea es escalar de forma balanceada tres dimensiones de la red:

- **Profundidad (d):** Número de capas.
- **Anchura (w):** Número de canales en cada capa.
- **Resolución (r):** Tamaño de la imagen de entrada.

### 🔬 Escalado Compuesto

El escalado se controla con un factor φ (phi) y tres coeficientes α, β, γ:

```
depth = α^φ
width = β^φ  
resolution = γ^φ
```

**Restricción:** α · β² · γ² ≈ 2, donde α ≥ 1, β ≥ 1, γ ≥ 1

### 🎯 Ventajas de EfficientNet:

1. **Eficiencia computacional**: Mejor accuracy con menos parámetros
2. **Escalado balanceado**: No solo aumenta una dimensión
3. **Transferible**: Funciona bien en diferentes tareas
4. **Arquitectura optimizada**: Basada en Neural Architecture Search (NAS)

## Arquitectura de EfficientNet

La siguiente imagen muestra la arquitectura y funcionamiento de EfficientNet:

![EfficientNet Architecture](IMAGE.png)

*Figura: Arquitectura y escalado compuesto de EfficientNet mostrando cómo se balancean las dimensiones de profundidad, anchura y resolución.*

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
from pathlib import Path

## 📊 Funcionamiento Detallado del Modelo EfficientNet

### 🏗️ **Arquitectura General**

EfficientNet sigue esta estructura principal:

```
Input Image → Stem → MBConv Blocks → Head → Output
```

### 🔍 **1. Stem (Inicio)**
- **Función**: Preprocesamiento inicial de la imagen
- **Operación**: Convolución 3x3 con stride=2
- **Salida**: Reduce resolución a la mitad, extrae características básicas
- **Canales**: RGB (3) → 32 características

### 🧱 **2. Bloques MBConv (Corazón del modelo)**

Los bloques MBConv son la innovación clave. Cada bloque realiza:

#### **Paso 1: Expansión (1x1 Conv)**
```python
# Si expand_ratio > 1
channels: in_ch → in_ch * expand_ratio
# Ejemplo: 32 → 192 canales (expand_ratio=6)
```

#### **Paso 2: Depthwise Convolution**
```python
# Convolución por grupos (cada canal por separado)
- Reduce parámetros significativamente
- Captura patrones espaciales por canal
- Kernel: 3x3 o 5x5
```

#### **Paso 3: Squeeze-and-Excitation (SE)**
```python
# Recalibración de canales
Global Average Pool → FC → ReLU → FC → Sigmoid → Multiply
```

#### **Paso 4: Proyección (1x1 Conv)**
```python
# Reducción de canales
channels: expanded → out_ch
# Ejemplo: 192 → 40 canales
```

#### **Paso 5: Skip Connection**
```python
if stride == 1 and in_ch == out_ch:
    output = projection + input  # Residual connection
```

### 🎯 **3. Head (Final)**
- **Global Average Pooling**: Convierte feature maps a vector
- **Dropout**: Regularización para evitar overfitting  
- **Linear**: Clasificación final (1000 clases en ImageNet)

### ⚡ **Flujo de Datos Completo**

```
Imagen (224x224x3)
    ↓ Stem Conv3x3
Feature Maps (112x112x32)
    ↓ MBConv Stage 1 (16 canales)
Feature Maps (112x112x16)
    ↓ MBConv Stage 2 (24 canales)
Feature Maps (56x56x24)
    ↓ MBConv Stage 3 (40 canales)
Feature Maps (28x28x40)
    ↓ MBConv Stage 4 (80 canales)
Feature Maps (14x14x80)
    ↓ MBConv Stage 5 (112 canales)
Feature Maps (14x14x112)
    ↓ MBConv Stage 6 (192 canales)
Feature Maps (7x7x192)
    ↓ MBConv Stage 7 (320 canales)
Feature Maps (7x7x320)
    ↓ Head Conv1x1
Feature Maps (7x7x1280)
    ↓ Global Average Pool
Vector (1280)
    ↓ Dropout + Linear
Logits (1000 clases)
```

## 2. Bloque Squeeze-and-Excitation (SE) - Explicación Detallada

### 🧠 **¿Qué problema resuelve SE?**

Las redes convolucionales tradicionales tratan todos los canales por igual. Sin embargo, **algunos canales contienen información más relevante que otros**. El bloque SE aprende a **recalibrar automáticamente** la importancia de cada canal.

### ⚙️ **Funcionamiento paso a paso:**

#### **Paso 1: Squeeze (Compresión) 🗜️**
```python
# Global Average Pooling
input: (Batch, Channels, Height, Width)
output: (Batch, Channels, 1, 1)
```
- **Función**: Convierte cada mapa de características 2D en un único valor
- **Resultado**: Captura la "esencia" global de cada canal
- **Matemáticamente**: `z_c = (1/H*W) * Σ(x_c(i,j))`

#### **Paso 2: Excitation (Activación) ⚡**
```python
# Dos capas completamente conectadas
FC1: channels → channels//reduction  # Compresión
ReLU: Activación no lineal
FC2: channels//reduction → channels   # Expansión  
Sigmoid: Normalización [0,1]
```
- **Función**: Aprende las interdependencias entre canales
- **Reduction**: Típicamente 4 o 16 (reduce parámetros)
- **Sigmoid**: Asegura que los pesos estén entre 0 y 1

#### **Paso 3: Scale (Reescalado) 📏**
```python
# Multiplicación elemento por elemento
output = input * se_weights.unsqueeze(-1).unsqueeze(-1)
```
- **Función**: Aplica los pesos aprendidos a cada canal
- **Resultado**: Canales importantes se amplifican, irrelevantes se atenúan

### 🎯 **Ejemplo Práctico:**
Si tenemos una imagen de un gato:
- **Canales de bordes**: Peso alto (importante para detectar forma)
- **Canales de textura**: Peso medio (útil para pelaje)
- **Canales de ruido**: Peso bajo (información irrelevante)

### 📈 **Beneficios:**
1. **Mejora accuracy** sin costo computacional significativo
2. **Atención automática** en características relevantes  
3. **Fácil integración** en cualquier arquitectura CNN
4. **Pocos parámetros adicionales** (~2% del total)

In [ ]:

class SEModule(nn.Module):
    def __init__(self, channels, reduction=4):
        super().__init__()
        hidden = max(1, channels // reduction)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(channels, hidden, kernel_size=1),
            nn.SiLU(inplace=True),
            nn.Conv2d(hidden, channels, kernel_size=1),
            nn.Sigmoid()
        )
    def forward(self, x):
        w = self.pool(x)
        w = self.fc(w)
        return x * w


## 3. Bloque MBConv (Mobile Inverted Bottleneck) - Análisis Completo

### 🏗️ **Filosofía del Diseño**

El bloque MBConv invierte la filosofía tradicional de los bottlenecks:
- **Bottleneck tradicional**: Ancho → Estrecho → Ancho
- **Inverted Bottleneck**: Estrecho → Ancho → Estrecho

### 🔄 **Arquitectura Detallada:**

```
Input (low-dim) → Expand (high-dim) → Filter → Compress (low-dim) → Output
```

#### **🚀 Ventaja 1: Expansión**
```python
# Expansión 1x1 (si expand_ratio > 1)
if expand_ratio != 1:
    channels: in_ch → in_ch * expand_ratio
```
- **Por qué**: Las convoluciones depthwise funcionan mejor con más canales
- **Ejemplo**: 32 canales → 192 canales (expand_ratio=6)
- **Beneficio**: Más "espacio" para aprender características complejas

#### **🎯 Ventaja 2: Depthwise Convolution**
```python
# Convolución por grupos (cada canal independiente)
groups = input_channels  # Cada canal se procesa por separado
```

**Comparación de parámetros:**
```python
# Convolución tradicional 3x3:
params_traditional = in_ch * out_ch * 3 * 3
# Ejemplo: 192 * 192 * 9 = 331,776 parámetros

# Depthwise + Pointwise:
params_depthwise = in_ch * 3 * 3 + in_ch * out_ch
# Ejemplo: 192 * 9 + 192 * 32 = 1,728 + 6,144 = 7,872 parámetros
# ¡42x menos parámetros!
```

#### **🧠 Ventaja 3: Información Preservada**
La conexión residual preserva información:
```python
if stride == 1 and in_ch == out_ch:
    output = processed_input + input  # Skip connection
```

### 📊 **Flujo Completo de un MBConv:**

```python
# Entrada: (B, 32, 56, 56)
#     ↓ Expand 1x1 (32→192)
# (B, 192, 56, 56)
#     ↓ Depthwise 3x3
# (B, 192, 28, 28)  # Si stride=2
#     ↓ SE Module (recalibración)
# (B, 192, 28, 28)
#     ↓ Project 1x1 (192→24)
# (B, 24, 28, 28)
#     ↓ + Skip (si dimensiones coinciden)
# Output: (B, 24, 28, 28)
```

### ⚡ **Componentes Clave:**

1. **Expansión 1x1**: 
   - Aumenta canales para mejor representación
   - Activación: Swish/SiLU (mejor que ReLU)

2. **Depthwise Convolution**: 
   - Captura patrones espaciales
   - Eficiencia computacional extrema

3. **Squeeze-and-Excitation**: 
   - Atención en canales importantes
   - Mejora significativa en accuracy

4. **Proyección 1x1**: 
   - Reduce dimensionalidad
   - Sin activación (preserva información)

5. **Skip Connection**: 
   - Evita degradación del gradiente
   - Permite redes más profundas

### 🎯 **Configuraciones por Etapa:**

| Etapa | Output Ch | Kernel | Stride | Expand | Repeticiones |
|-------|-----------|--------|--------|---------|-------------|
| 1     | 16        | 3      | 1      | 1       | 1          |
| 2     | 24        | 3      | 2      | 6       | 2          |
| 3     | 40        | 5      | 2      | 6       | 2          |
| 4     | 80        | 3      | 2      | 6       | 3          |
| 5     | 112       | 5      | 1      | 6       | 3          |
| 6     | 192       | 5      | 2      | 6       | 4          |
| 7     | 320       | 3      | 1      | 6       | 1          |

In [ ]:

class MBConv(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size=3, stride=1,
                 expand_ratio=6, se_ratio=0.25, drop_rate=0.0):
        super().__init__()
        self.use_res = (stride == 1 and in_ch == out_ch)
        self.drop_rate = drop_rate
        mid_ch = in_ch if expand_ratio == 1 else in_ch * expand_ratio
        
        layers = []
        if expand_ratio != 1:
            layers += [
                nn.Conv2d(in_ch, mid_ch, kernel_size=1, bias=False),
                nn.BatchNorm2d(mid_ch),
                nn.SiLU(inplace=True)
            ]
        
        layers += [
            nn.Conv2d(mid_ch, mid_ch, kernel_size, stride, kernel_size//2, groups=mid_ch, bias=False),
            nn.BatchNorm2d(mid_ch),
            nn.SiLU(inplace=True)
        ]
        
        self.features = nn.Sequential(*layers)
        self.se = SEModule(mid_ch, reduction=int(1/se_ratio))
        self.project = nn.Sequential(
            nn.Conv2d(mid_ch, out_ch, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_ch)
        )
    
    def forward(self, x):
        identity = x
        y = self.features(x)
        y = self.se(y)
        y = self.project(y)
        if self.use_res:
            y = y + identity
        return y



## 4. EfficientNet-B0 Mini (Educativo)

Implementación reducida de EfficientNet-B0, útil para fines educativos.  
No incluye pesos preentrenados, pero mantiene la estructura principal.


### 🏆 **Innovaciones Clave de EfficientNet**

#### **1. Neural Architecture Search (NAS)**
- **Automatización**: La arquitectura base se diseñó usando búsqueda automática
- **Optimización**: Balance óptimo entre accuracy y eficiencia
- **Resultado**: EfficientNet-B0 como arquitectura base optimal

#### **2. Compound Scaling Method**
```python
# Escalado tradicional (problemático):
# Solo profundidad: ResNet-50 → ResNet-101 → ResNet-152
# Solo anchura: Más canales por capa
# Solo resolución: Imágenes más grandes

# Escalado compuesto (EfficientNet):
depth = α^φ     # α = 1.2
width = β^φ     # β = 1.1  
resolution = γ^φ # γ = 1.15
# Restricción: α · β² · γ² ≈ 2
```

#### **3. Comparación con Otras Arquitecturas**

| Modelo | Parámetros | FLOPs | Top-1 Accuracy |
|--------|------------|-------|---------------|
| ResNet-50 | 25.6M | 4.1B | 76.0% |
| ResNet-152 | 60.2M | 11.6B | 77.8% |
| DenseNet-264 | 33.3M | 5.8B | 77.9% |
| **EfficientNet-B0** | **5.3M** | **0.39B** | **77.1%** |
| **EfficientNet-B7** | **66M** | **37B** | **84.4%** |

### 🔬 **Análisis de Eficiencia**

#### **Memoria y Computación:**
```python
# Comparación de operaciones (ejemplo 224x224):
Traditional Conv 3x3: O(H×W×Cin×Cout×9)
Depthwise + Pointwise: O(H×W×Cin×9 + H×W×Cin×Cout)

# Para 192 canales entrada/salida:
Traditional: 224×224×192×192×9 = 52.6B ops
MBConv: 224×224×192×9 + 224×224×192×192 = 0.87B ops
# ¡60x más eficiente!
```

#### **Escalado Inteligente:**
- **α (profundidad)**: Más capas → mejor representación
- **β (anchura)**: Más canales → mayor capacidad  
- **γ (resolución)**: Imágenes grandes → más detalles finos
- **Balance**: Los tres factores se complementan exponencialmente

In [ ]:

class EfficientNetB0Mini(nn.Module):
    def __init__(self, num_classes=1000, drop_rate=0.2):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.SiLU(inplace=True)
        )
        
        cfg = [
            (16, 3, 1, 1, 1),
            (24, 3, 2, 6, 2),
            (40, 5, 2, 6, 2),
            (80, 3, 2, 6, 3),
            (112, 5, 1, 6, 3),
            (192, 5, 2, 6, 4),
            (320, 3, 1, 6, 1)
        ]
        
        blocks = []
        in_ch = 32
        for out_ch, k, s, exp, reps in cfg:
            for i in range(reps):
                stride = s if i == 0 else 1
                blocks.append(MBConv(in_ch, out_ch, kernel_size=k, stride=stride, expand_ratio=exp))
                in_ch = out_ch
        
        self.blocks = nn.Sequential(*blocks)
        self.head = nn.Sequential(
            nn.Conv2d(in_ch, 1280, kernel_size=1, bias=False),
            nn.BatchNorm2d(1280),
            nn.SiLU(inplace=True),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Dropout(drop_rate),
            nn.Linear(1280, num_classes)
        )
    
    def forward(self, x):
        x = self.stem(x)
        x = self.blocks(x)
        x = self.head(x)
        return x


In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"
model = EfficientNetB0Mini().to(device)
print("Modelo creado en", device)



## 5. Inferencia

Ejemplo de inferencia con una imagen de entrada.


In [ ]:

transform_infer = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

img_path = "foto.jpg"
if Path(img_path).exists():
    img = Image.open(img_path).convert("RGB")
    x = transform_infer(img).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        logits = model(x)
        probs = logits.softmax(dim=1)
    print("Shape de salida:", probs.shape)
else:
    print("Coloca una imagen llamada 'foto.jpg' en el directorio actual.")
